In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from xgboost import XGBRegressor
#from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Imputer
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

train_file_path = '../input/train.csv'
test_file_path = '../input/test.csv'
subm_sample_file_path = '../input/sample_submission.csv'

train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)
sample_submission = pd.read_csv(subm_sample_file_path)

my_pipeline = make_pipeline(Imputer(), GradientBoostingRegressor())

In [ ]:
y_train = train_data['SalePrice']
del train_data['SalePrice']

#predictors = ['FullBath', 'BedroomAbvGr','LotArea', 'YearBuilt', 'GarageCars','Fireplaces', 'OverallCond', 'KitchenAbvGr', 'GrLivArea']
#x_train = train_data[predictors]
#X_test = test_data[predictors]
x_train = train_data
X_test = test_data
#x_train.columns

In [ ]:
# One hot encoding for categorical data.
# this should be done before imputation

one_hot_encoded_training = pd.get_dummies(x_train)
one_hot_encoded_test = pd.get_dummies(X_test)
x_train_OHE, X_test = one_hot_encoded_training.align(one_hot_encoded_test,
                                                                    join='left', 
                                                                    axis=1)



In [ ]:
#this function splits the training data to generate some cross validation data
#X_train, X_val, Y_train, Y_val = train_test_split(x_train_OHE, y_train, train_size=0.3)

#cross val performed without train_test_split
scores = cross_val_score(my_pipeline, x_train_OHE, y_train, scoring='neg_mean_absolute_error')
print(scores)
print('Mean Absolute Error %2f' %(-1 * scores.mean()))

#X_train_imputed_plus = X_train.copy()
#X_val_imputed_plus = X_val.copy()

In [ ]:
my_pipeline.fit(x_train_OHE, y_train)
predicted_prices = my_pipeline.predict(X_test)
#submission file
my_submission = pd.DataFrame({'Id': test_data.Id, 'SalePrice': predicted_prices})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

In [ ]:
#Replacing all Nans with a versy small number

###########################################################
# DONT REPLACE WITH JUST 0 as this is an int? (use 0. OR put .astype(np.float32)) as below

#X_test_replace = x_test.fillna(0.).astype(np.float32)

##############################################################


#make sure no training examples went missing (check count)
#X_test_replace.describe()

In [ ]:
#scoredataset function, takend from Dan B's tute
#def score_dataset(X_train, X_test, y_train, y_test):
#    model = RandomForestRegressor()
#    model.fit(X_train, y_train)
#    preds = model.predict(X_test)
#    return mean_absolute_error(y_test, preds)

# imputing variables, code taken from Dan Becker's tutorial

#my_imputer = Imputer()
#X_train_imputed = my_imputer.fit_transform(X_train)
#X_val_imputed = my_imputer.transform(X_val)
#X_test_imputed = my_imputer.transform(X_test)
#print("Mean Absolute Error from Imputation:")
#print(score_dataset(X_train_imputed, X_val_imputed, Y_train, Y_val))

In [ ]:
#cols_with_missing = (col for col in X_train.columns 
#                                 if X_train[col].isnull().any())
#for col in cols_with_missing:
#    X_train_imputed_plus[col + '_was_missing'] = X_train_imputed_plus[col].isnull()
#    X_val_imputed_plus[col + '_was_missing'] = X_val_imputed_plus[col].isnull()

# Imputation
#my_imputer = Imputer()
#X_train_imputed_plus = my_imputer.fit_transform(X_train_imputed_plus)
#X_val_imputed_plus = my_imputer.transform(X_val_imputed_plus)

#print("Mean Absolute Error from Imputation while Track What Was Imputed:")
#print(score_dataset(X_train_imputed_plus, X_val_imputed_plus, Y_train, Y_val))

In [ ]:
# these next few lines were using a decision tree fucntion
#my_model = DecisionTreeRegressor(max_leaf_nodes=42)
#my_model.fit(X_train, Y_train)




In [ ]:
#predictions = my_model.predict(X_val)
#mean_absolute_error(Y_val, predictions)

In [ ]:
#function written by Dan Becker to calculate mean absolute error based on number
# of max leaf nodes selected.

#def get_mae(max_leaf_nodes, predictors_train, predictors_val, targ_train, targ_val):
#    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
#    model.fit(predictors_train, targ_train)
#    preds_val = model.predict(predictors_val)
#    mae = mean_absolute_error(targ_val, preds_val)
#    return(mae)

#runs a for loop that tries different ranges of max leaf nodes and outputs each values
# MAE score, to determine most effective option

#listNumbers = list(range(40, 50))

#for max_leaf_nodes in listNumbers:
#    my_mae = get_mae(max_leaf_nodes, X_train, X_val, Y_train, Y_val)
#    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

In [ ]:
# uses the random forest regressor model from scikit learn. 
# Random Forest is gives better results thean the decision tree model

#forest_model = RandomForestRegressor()
#forest_model.fit(X_train_imputed, Y_train)
#forest_predictions = forest_model.predict(X_val_imputed)
#print(mean_absolute_error(Y_val, forest_predictions))

In [ ]:
# XGBoost

#my_model = XGBRegressor(#n_estimators=1000, #learning_rate=0.05)
#my_model = GradientBoostingRegressor()
# Add silent=True to avoid printing out updates with each cycle
#my_model.fit(X_train_imputed, Y_train, early_stopping_rounds=5, eval_set=[(X_val_imputed, Y_val)], verbose=False)


In [ ]:
#using these partial dependency graphs, with gradientboostregressor,
# we can determine which features have a larger effect on the trining of the model
# as seen below, overall quality is much more significant to the price of a house than
# lotarea is.

#my_model.fit(X_train_imputed, Y_train)
# Here we make the plot
#my_plots = plot_partial_dependence(my_model,       
#                                   features=[ 0, 2], # column numbers of plots we want to show
#                                   X=X_train_imputed,            # raw predictors data.
#                                   feature_names=['LotArea', 'YearBuilt', 'OverallQual'], # labels on graphs
#                                   grid_resolution=10) # number of values to plot on x axis

In [ ]:
#predicted_prices = forest_model.predict(X_test_imputed)
#predicted_prices = my_model.predict(X_val_imputed)
#print(predicted_prices)
#print("Mean Absolute Error : " + str(mean_absolute_error(predicted_prices, Y_val)))

In [ ]:
#predicted_prices = my_model.predict(X_test_imputed)
#submission file
#my_submission = pd.DataFrame({'Id': test_data.Id, 'SalePrice': predicted_prices})
# you could use any filename. We choose submission here
#my_submission.to_csv('submission.csv', index=False)